# Predictor Validation and Tuning

## Environment Setup

In [ ]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import train_test_split  # type: ignore
from sklearn.linear_model import LinearRegression  # type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import time
import matplotlib.pyplot as plt

import json

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import dask.dataframe as dd
import os

def get_dataset_name(M, efC, efS, query_num, ds_name, k, logint):
    return f"/data/mchatzakis/et_training_data/early-stop-training/{ds_name}/k{k}/M{M}_efC{efC}_efS{efS}_qs{query_num}_li{logint}.txt"

def get_validation_dataset_name(M, efC, efS, query_num, ds_name, k, logint): 
    return f"../../experiments/results/validation_logging/{ds_name}/k{k}/M{M}_efC{efC}_efS{efS}_qs{query_num}_li{logint}.txt"

def get_model_name(M, ef, s, ds_name, k, logint, selected_features, n_estimators=100):
    return f"../../predictor_models/lightgbm/{ds_name}_M{M}_ef{ef}_s{s}_k{k}_nestim{n_estimators}_li{logint}_{selected_features}.txt"

def compute_P99(y_true, y_pred):
    y_diff = np.abs(y_true - y_pred)
    return np.percentile(y_diff, 99)

def compute_P1(y_true, y_pred):
    y_diff = np.abs(y_true - y_pred)
    return np.percentile(y_diff, 1)

def get_interval_tuning_dataset_name(M, efC, efS, s, ds_name, k, r_target, ipi, mpi, mode="search"):
    return f"../../experiments/results/interval-tuning/{mode}/{ds_name}/k{k}/M{M}_efC{efC}_efS{efS}_qs{s}_tr{r_target:.2f}_ipi{ipi}_mpi{mpi}.txt"

def get_no_early_stop_dataset_name(M, efC, efS, s, ds_name, k):
    return f"../../experiments/results/no-early-stop/validation/{ds_name}/k{k}/M{M}_efC{efC}_efS{efS}_qs{s}.txt"

dataset_params = {
        "SIFT100M": {
            "M": 32,
            "efC": 500,
            "efS": 500,
            "li": 1,
            "label": "SIFT100M",
        },
        "GIST1M": {
            "M": 32,
            "efC": 500,
            "efS": 1000,
            "li": 1,
            "label": "GIST1M",
        },
        "GLOVE100": {
            "M": 16,
            "efC": 500,
            "efS": 500,
            "li": 1,
            "label": "GLOVE1M",
        },
        "DEEP100M":{
            "M": 32,
            "efC": 500,
            "efS": 750,
            "li": 1,
            "label": "DEEP100M",
        },
        "T2I100M":{
            "M": 80,
            "efC": 1000,
            "efS": 2500,
            "li": 2,
            "label": "T2I100M",
        }
    }

SEED = 42

n_estimators = 100

index_metric_feats = ["step", "dists", "inserts"]
neighbor_distances_feats = ["first_nn_dist", "nn_dist", "furthest_dist"]
neighbor_stats_feats = ["avg_dist", "variance", "percentile_25", "percentile_50", "percentile_75"]
all_feats = index_metric_feats + neighbor_distances_feats + neighbor_stats_feats

columns_to_load = ["qid", "elaps_ms"] + all_feats + ["r", "feats_collect_time_ms"]
    
model_conf = {
    "lightgbm": lgb.LGBMRegressor(objective='regression', random_state=SEED, n_estimators=n_estimators, verbose = -1),
}

feature_classes = {
    "index_metric_feats": index_metric_feats,
    "neighbor_distances_feats": neighbor_distances_feats,
    "neighbor_stats_feats": neighbor_stats_feats,
    "index_metrics_and_neighbor_distances": index_metric_feats + neighbor_distances_feats,
    "index_metrics_and_neighbor_stats": index_metric_feats + neighbor_stats_feats,
    "neighbor_distances_and_neighbor_stats": neighbor_distances_feats + neighbor_stats_feats,
    "all_feats": all_feats,
}

print(columns_to_load)

PLOTS_DIR = "./../../experiments/revision-plots/"

## Early Termination Opportunities

In [ ]:
ds_name = "SIFT100M"
M = dataset_params[ds_name]["M"]
efC = dataset_params[ds_name]["efC"]
efS = dataset_params[ds_name]["efS"]
k=100
li=1

In [ ]:
dask_data_all = dd.read_csv(get_dataset_name(M, efC, efS, 10000, ds_name, k, li), usecols=columns_to_load)
data_all = dask_data_all.compute()
print(data_all.shape)

total_queries = 2000
metric = "elaps_ms"

queries = [] # List of [time, recall]
for qid in range(total_queries):
    curr_query = data_all[data_all["qid"] == qid]
    recalls, times = curr_query["r"], curr_query[metric]
    queries.append([times, recalls])

# Define summaries for each query
summarized_queries = []
for times, recalls in queries:
    # Compute mean of times and recalls
    mean_time = np.mean(times)
    mean_recall = np.mean(recalls)
    summarized_queries.append([mean_time, mean_recall])

# Convert summarized queries to a NumPy array
summarized_queries_array = np.array(summarized_queries)

# Normalize the data
scaler = StandardScaler()
normalized_queries = scaler.fit_transform(summarized_queries_array)

# Clustering with 20 clusters
num_queries_to_plot = 4
kmeans = KMeans(n_clusters=num_queries_to_plot, random_state=SEED)
labels = kmeans.fit_predict(normalized_queries)

# Select one query from each cluster
selected_queries = []
for cluster_id in range(num_queries_to_plot):
    cluster_queries = [queries[i] for i in range(len(queries)) if labels[i] == cluster_id]
    selected_queries.append(cluster_queries[0])  # Choose the first query in each cluster

# save selected queries
selected_queries_dict = {}
for i, query in enumerate(selected_queries):
    times, recalls = query
    selected_queries_dict[i] = {"times": times.tolist(), "recalls": recalls.tolist()}

print(selected_queries_dict)

with open(f"../../experiments/generated_json/selected_queries.json", "w") as f:
    json.dump(selected_queries_dict, f)

In [ ]:
import matplotlib.pyplot as plt
import json

selected_queries = []
selected_queries_dict = {}
with open(f"../../experiments/generated_json/selected_queries.json", "r") as f:
    selected_queries_dict = json.load(f)

for i in range(len(selected_queries_dict)):
    query = selected_queries_dict[str(i)]
    times, recalls = query["times"], query["recalls"]
    selected_queries.append([times, recalls])

plt.rcParams.update({"font.size": 30})

# Plot the selected queries
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

colors = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]
markers = ["o", "s", "D", "v", "^", ">", "<", "p", "P", "*"]

# Determine the x-axis limits
max_time = max([max(query[0]) / 1000 for query in selected_queries])

# Fill the background above y=0.8 with light gray
ax.fill_between(
    [-1, 15],  # x-values covering the whole range
    0.8, 1.1,  # y-range for the fill
    color='lightgray', alpha=0.5  # fill color and transparency
)

plt.axhline(y=0.8, color='black', linestyle='--', alpha=0.8, linewidth=2)

query_num = 0
for i, query in enumerate(selected_queries):
    
    times, recalls = query
    times = list(times)
    recalls = list(recalls)
    times = [t / 1000 for t in times]

    if colors[i] == "green":
        continue

    # Find the index where recall reaches 0.8
    index = -1
    for j, r in enumerate(recalls):
        if r >= 0.80:
            plt.axvline(
                x=times[j], color=colors[i], linestyle=":", ymax=r - 0.06, linewidth=2, alpha=0.8, ymin=0.045
            )
            index = j
            break

    # Downsample the data
    downsample_factor = 1200
    downsampled_times = []
    downsampled_recalls = []
    recalls_to_put_markers = [0.2, 0.4, 0.6, 0.8, 1]
    already_found_recalls = [False] * len(recalls_to_put_markers)
    marker_plot_times = []
    marker_plot_recalls = []

    for j, r in enumerate(recalls):
        if r in recalls_to_put_markers and not already_found_recalls[recalls_to_put_markers.index(r)]:
            marker_plot_times.append(times[j])
            marker_plot_recalls.append(r)
            already_found_recalls[recalls_to_put_markers.index(r)] = True
            downsampled_times.append(times[j])
            downsampled_recalls.append(r)
        if j == len(recalls) - 1:
            marker_plot_times.append(times[j])
            marker_plot_recalls.append(r)

    if downsampled_times and downsampled_times[0] != 0:
        downsampled_times.append(times[-1])
        downsampled_recalls.append(recalls[-1])
        downsampled_times.insert(0, 0)
        downsampled_recalls.insert(0, 0)
        marker_plot_times.append(0)
        marker_plot_recalls.append(0)

    # Split the data into before and after recall 0.8
    before_08_times = [t for t, r in zip(downsampled_times, downsampled_recalls) if r <= 0.8]
    before_08_recalls = [r for r in downsampled_recalls if r <= 0.8]
    after_08_times = [t for t, r in zip(downsampled_times, downsampled_recalls) if r >= 0.8]
    after_08_recalls = [r for r in downsampled_recalls if r >= 0.8]

    # Plot the lines with different transparency
    query_num += 1
    ax.plot(before_08_times, before_08_recalls, linestyle="-", alpha=0.9, color=colors[i], linewidth=3)
    ax.plot(after_08_times, after_08_recalls, linestyle="--", alpha=0.7, color=colors[i], linewidth=3)

    # Split the markers
    marker_before_08_times = [t for t, r in zip(marker_plot_times, marker_plot_recalls) if r <= 0.8]
    marker_before_08_recalls = [r for r in marker_plot_recalls if r <= 0.8]
    marker_after_08_times = [t for t, r in zip(marker_plot_times, marker_plot_recalls) if r >= 0.8]
    marker_after_08_recalls = [r for r in marker_plot_recalls if r >= 0.8]

    # Plot the scatter points for recall markers
    ax.scatter(marker_before_08_times, marker_before_08_recalls, color=colors[i], s=300, zorder=5, alpha=0.9, marker=markers[i])
    ax.scatter(marker_after_08_times, marker_after_08_recalls, color=colors[i], s=300, alpha=0.4, marker=markers[i])

    # Add legend for each line including the marker:
    ax.plot([], [], '-', label=f"Query {query_num}", color=colors[i], linewidth=3, marker=markers[i], markersize=20)

#lines_for_legend = [line for line in ax.get_lines() if not line.get_label().startswith('_')]

# Add custom legend for the "Wasted Effort" area
#custom_legend = [plt.Line2D([0], [0], color='lightgray', lw=10, alpha=0.5)]
#ax.legend(custom_legend + lines_for_legend, ['Wasted Effort'] + [line.get_label() for line in lines_for_legend], loc="lower right")
ax.text(max_time * 1.02, 0.85, 'Wasted\n  Effort', fontsize=28, color='black', bbox=dict(facecolor='lightgray', alpha=0.0))

ax.set_xlabel("Search Time (ms)")
ax.set_ylabel("Recall")
ax.legend(loc="lower right") 

#lock x axis to 0-14
ax.set_xlim(-0.5, 14)
ax.set_ylim(-0.08, 1.1)

ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

fig.tight_layout()
plt.show()

# Save as PDF
fig.savefig(f"./{PLOTS_DIR}/early_termination_opportunities_test3.pdf", bbox_inches='tight')
print(f"Fig saved at ./{PLOTS_DIR}/early_termination_opportunities_test3.pdf")

In [ ]:
import matplotlib.pyplot as plt
import json

selected_queries = []
selected_queries_dict = {}
with open(f"../../experiments/generated_json/selected_queries.json", "r") as f:
    selected_queries_dict = json.load(f)

for i in range(len(selected_queries_dict)):
    query = selected_queries_dict[str(i)]
    times, recalls = query["times"], query["recalls"]
    selected_queries.append([times, recalls])

plt.rcParams.update({"font.size": 30})

# Plot the selected queries
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

colors = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]
markers = ["o", "s", "D", "v", "^", ">", "<", "p", "P", "*"]

# Determine the x-axis limits
max_time = max([max(query[0]) / 1000 for query in selected_queries])

# Fill the background above y=0.8 with light gray
ax.fill_between(
    [-1, 15],  # x-values covering the whole range
    0.8, 1.1,  # y-range for the fill
    color='lightgray', alpha=0.5  # fill color and transparency
)

plt.axhline(y=0.8, color='black', linestyle='--', alpha=0.8, linewidth=2)

query_num = 0
for i, query in enumerate(selected_queries):
    
    #if (i != 0 and i != 1):
    #    continue
    
    times, recalls = query
    times = list(times)
    recalls = list(recalls)
    times = [t / 1000 for t in times]

    if colors[i] == "green":
        continue

    # Find the index where recall reaches 0.8
    index = -1
    for j, r in enumerate(recalls):
        if r >= 0.80:
            plt.axvline(
                x=times[j], color=colors[i], linestyle=":", ymax=r - 0.06, linewidth=2, alpha=0.8, ymin=0.045
            )
            index = j
            break

    # Downsample the data
    downsample_factor = 1200
    downsampled_times = []
    downsampled_recalls = []
    recalls_to_put_markers = [0.2, 0.4, 0.6, 0.8, 1]
    already_found_recalls = [False] * len(recalls_to_put_markers)
    marker_plot_times = []
    marker_plot_recalls = []

    for j, r in enumerate(recalls):
        if r in recalls_to_put_markers and not already_found_recalls[recalls_to_put_markers.index(r)]:
            marker_plot_times.append(times[j])
            marker_plot_recalls.append(r)
            already_found_recalls[recalls_to_put_markers.index(r)] = True
            downsampled_times.append(times[j])
            downsampled_recalls.append(r)
        if j == len(recalls) - 1:
            marker_plot_times.append(times[j])
            marker_plot_recalls.append(r)

    if downsampled_times and downsampled_times[0] != 0:
        downsampled_times.append(times[-1])
        downsampled_recalls.append(recalls[-1])
        downsampled_times.insert(0, 0)
        downsampled_recalls.insert(0, 0)
        marker_plot_times.append(0)
        marker_plot_recalls.append(0)

    # Split the data into before and after recall 0.8
    before_08_times = [t for t, r in zip(downsampled_times, downsampled_recalls) if r <= 0.8]
    before_08_recalls = [r for r in downsampled_recalls if r <= 0.8]
    after_08_times = [t for t, r in zip(downsampled_times, downsampled_recalls) if r >= 0.8]
    after_08_recalls = [r for r in downsampled_recalls if r >= 0.8]

    # Plot the lines with different transparency
    query_num += 1
    ax.plot(before_08_times, before_08_recalls, linestyle="-", alpha=0.9, color=colors[i], linewidth=3)
    ax.plot(after_08_times, after_08_recalls, linestyle="--", alpha=0.7, color=colors[i], linewidth=3)

    # Split the markers
    marker_before_08_times = [t for t, r in zip(marker_plot_times, marker_plot_recalls) if r <= 0.8]
    marker_before_08_recalls = [r for r in marker_plot_recalls if r <= 0.8]
    marker_after_08_times = [t for t, r in zip(marker_plot_times, marker_plot_recalls) if r >= 0.8]
    marker_after_08_recalls = [r for r in marker_plot_recalls if r >= 0.8]

    # Plot the scatter points for recall markers
    ax.scatter(marker_before_08_times, marker_before_08_recalls, color=colors[i], s=300, zorder=5, alpha=0.9, marker=markers[i])
    ax.scatter(marker_after_08_times, marker_after_08_recalls, color=colors[i], s=300, alpha=0.4, marker=markers[i])

    # Add legend for each line including the marker:
    ax.plot([], [], '-', label=f"Query {query_num}", color=colors[i], linewidth=3, marker=markers[i], markersize=20)

#lines_for_legend = [line for line in ax.get_lines() if not line.get_label().startswith('_')]

# Add custom legend for the "Wasted Effort" area
#custom_legend = [plt.Line2D([0], [0], color='lightgray', lw=10, alpha=0.5)]
#ax.legend(custom_legend + lines_for_legend, ['Wasted Effort'] + [line.get_label() for line in lines_for_legend], loc="lower right")
#ax.text(max_time * 1.02, 0.85, 'Wasted\n  Effort', fontsize=28, color='black', bbox=dict(facecolor='lightgray', alpha=0.0))

ax.set_xlabel("Search Time (ms)")
ax.set_ylabel("Recall")
ax.legend(loc="lower right") 

#lock x axis to 0-14
ax.set_xlim(-0.5, 14)
ax.set_ylim(-0.08, 1.1)

ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

fig.tight_layout()
plt.show()

# Save as PDF
fig.savefig(f"./{PLOTS_DIR}/early_termination_opportunities_presentation_3.png")#, bbox_inches='tight')
print(f"Fig saved at ./{PLOTS_DIR}/early_termination_opportunities_presentation_3.png")

## Evaluate the models

In [ ]:
results = {}
with open( "../../experiments/generated_json/predictor_validation_results.json", "r") as f:
    results = json.load(f)

all_training_queries = ["10000"]
all_k_values = ["10", "25", "50", "75", "100"]
all_datasets = ["GLOVE100", "SIFT100M", "GIST1M", "DEEP100M"]#, "T2I100M"]
#all_lis = ["1"]
all_metrics = ["r2", "mse", "mae"]
all_feature_types = feature_classes.keys()

best_models = {}

# For every feature class, create a list fo the metrics for each conf
#feature_scores = {}
#for feature_class in all_feature_types:
#    for li in all_lis:
#        feature_scores[f"{feature_class}_{li}"] = {}
#        for metric in all_metrics:
#            feature_scores[f"{feature_class}_{li}"][metric] = []

feature_scores = {}
for feature_class in all_feature_types:
    feature_scores[f"{feature_class}"] = {}
    for metric in all_metrics:
        feature_scores[f"{feature_class}"][metric] = []


#print(feature_scores)

for ds_name in all_datasets:
    best_models[ds_name] = {}
        
    for i, k in enumerate(all_k_values):        
        best_r2 = -1
        best_queries = -1
        best_li = -1
        best_p99 = -1
        best_mse = 1000000
        best_features = ""
        
        for j, metric in enumerate(all_metrics):
            #for li in all_lis:
            for selected_features in all_feature_types:#feature_classes.keys():
                    
                metric_values = [results[ds_name][k][s][li][selected_features][metric] for s in all_training_queries]                    
                queries, metrics = all_training_queries, metric_values
                    
                feature_scores[f"{selected_features}_{li}"][metric].append(metrics)

#print(feature_scores)

# Average the metrics for each feature class
feature_avg_scores = {}
for feature_class in all_feature_types:
    for li in all_lis:
        feature_avg_scores[f"{feature_class}_{li}"] = {}
        for metric in all_metrics:
            feature_avg_scores[f"{feature_class}_{li}"][metric] = np.mean(feature_scores[f"{feature_class}_{li}"][metric])

# Sort by min mse
sorted_feature_avg_scores = {k: v for k, v in sorted(feature_avg_scores.items(), key=lambda item: item[1]["mse"], reverse=False)}
sorted_feature_avg_scores

# Print the first 5
for i, (feature_class, metrics) in enumerate(sorted_feature_avg_scores.items()):
    #if i >= 5:
    #    break

    print(f"{feature_class}")
    for metric in all_metrics:
        if metric == "p99" or metric == "r2" or metric == "" or metric == "mae":
            print(f"    {metric}: {metrics[metric]:.2f}")
        else:
            print(f"    {metric}: {metrics[metric]:.3f}")

# Pick the best feature class and print it
best_feature_class = list(sorted_feature_avg_scores.keys())[0]
best_feature_class

In [ ]:
results = {}
with open( "../../experiments/generated_json/predictor_validation_results.json", "r") as f:
    results = json.load(f)

all_training_queries = ["10000"]
all_k_values = ["10", "25", "50", "75", "100"]
all_datasets = ["GLOVE100", "SIFT100M", "GIST1M", "DEEP100M"]
all_metrics = ["r2", "mse", "mae"]
all_feature_types = feature_classes.keys()

best_models = {}

feature_scores = {}
for feature_class in all_feature_types:
    feature_scores[f"{feature_class}"] = {}
    for metric in all_metrics:
        feature_scores[f"{feature_class}"][metric] = []


#print(feature_scores)

for ds_name in all_datasets:
    best_models[ds_name] = {}
        
    for i, k in enumerate(all_k_values):        
        best_r2 = -1
        best_queries = -1
        best_li = -1
        best_p99 = -1
        best_mse = 1000000
        best_features = ""
        
        for j, metric in enumerate(all_metrics):
            for selected_features in all_feature_types:
                li = str(dataset_params[ds_name]["li"])
                metric_values = [results[ds_name][k][s][li][selected_features][metric] for s in all_training_queries]                    
                queries, metrics = all_training_queries, metric_values
                    
                feature_scores[f"{selected_features}"][metric].append(metrics)

#print(feature_scores)

# Average the metrics for each feature class
feature_avg_scores = {}
for feature_class in all_feature_types:
    feature_avg_scores[f"{feature_class}"] = {}
    for metric in all_metrics:
        feature_avg_scores[f"{feature_class}"][metric] = np.mean(feature_scores[f"{feature_class}"][metric])

# Sort by min mse
sorted_feature_avg_scores = {k: v for k, v in sorted(feature_avg_scores.items(), key=lambda item: item[1]["mse"], reverse=False)}
sorted_feature_avg_scores

# Print the first 5
for i, (feature_class, metrics) in enumerate(sorted_feature_avg_scores.items()):
    #if i >= 5:
    #    break

    print(f"{feature_class}")
    for metric in all_metrics:
        if metric == "mse" or metric == "mae":
            print(f"    {metric}: {metrics[metric]:.4f}")
        else:
            print(f"    {metric}: {metrics[metric]:.2f}")

# Pick the best feature class and print it
best_feature_class = list(sorted_feature_avg_scores.keys())[0]
best_feature_class

In [ ]:
all_datasets = ["GLOVE100", "SIFT100M", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = ["10", "25", "50", "75", "100"]
best_feature_class = "all_feats"
#best_li = 1
best_s = 10000

for ds_name in all_datasets:
    training_results_dict = {}
    with open(f"../../experiments/generated_json/training_results_{ds_name}.json", "r") as f:
        training_results_dict = json.load(f)
    
    dataset_samples = 0
    dataset_train_time = 0
    
    for k in all_k_values:
        best_li = str(dataset_params[ds_name]["li"])
        training_time = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["training_time"]
        training_samples = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["training_data_size"]
        learning_rate = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["learning_rate"]
        feature_importances = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["feature_importances"]
        
        dataset_samples += training_samples
        dataset_train_time += training_time
    
    dataset_samples = dataset_samples / len(training_results_dict.keys())
    dataset_train_time = dataset_train_time / len(training_results_dict.keys())
    
    print(f"Dataset: {ds_name} => Samples: {dataset_samples:.0f}, Training Time: {dataset_train_time:.0f}s")
        

## Feature Importance

In [ ]:
best_feature_class = "all_feats"
best_s = 10000
all_datasets = ["GLOVE100", "SIFT100M", "GIST1M", "DEEP100M"]

all_k_values = ["10", "25", "50", "75", "100"]

feature_scores = {}
for ds_name in all_datasets:
    training_results_dict = {}
    with open(f"../../experiments/generated_json/training_results_{ds_name}.json", "r") as f:
        training_results_dict = json.load(f)
    
    for k in training_results_dict.keys():
        #print(training_results_dict[ds_name][k][str(best_s)][str(best_li)][best_feature_class])
        best_li = str(dataset_params[ds_name]["li"])
        training_time = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["training_time"]
        training_samples = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["training_data_size"]
        learning_rate = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["learning_rate"]
        feature_importances = training_results_dict[k][str(best_s)][str(best_li)][best_feature_class]["feature_importances"]
        for entry in feature_importances:
            feature_name = entry["Feature"]
            importance_score = entry["Importance"]
            
            if feature_name not in feature_scores:
                feature_scores[feature_name] = [importance_score]
            else:
                feature_scores[feature_name].append(importance_score)

feature_avg_scores = {}
for feature_name, scores in feature_scores.items():
    feature_avg_scores[feature_name] = np.mean(scores)

sorted_feature_avg_scores = {k: v for k, v in sorted(feature_avg_scores.items(), key=lambda item: item[1], reverse=True)}

#for i, (feature_name, score) in enumerate(sorted_feature_avg_scores.items()):
#    print(f">> {feature_name}: {score:.2f}")
    
# Transform to perc
total_score = sum(feature_avg_scores.values())
perc_feature_avg_scores = {k: v / total_score for k, v in feature_avg_scores.items()}
sorted_perc_feature_avg_scores = {k: v for k, v in sorted(perc_feature_avg_scores.items(), key=lambda item: item[1], reverse=True)}
# Print again:
for i, (feature_name, score) in enumerate(sorted_perc_feature_avg_scores.items()):
    print(f">> {feature_name}: {(score * 100):.0f}%")

## Visualizations for the best models

In [ ]:
import matplotlib.pyplot as plt
import os

plt.rcParams.update({"font.size": 42})

# Mapping for metric axis titles
metric_axis_titles = {
    "mae": r"$MAE$",
    "mse": r"$MSE$",
    "r2": r"$R^2$",
    "p99": r"$P99$"
}

all_training_queries = ["100", "1000", "5000", "10000", "15000", "20000"]
all_k_values = ["10", "25", "50", "75", "100"]
all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_metrics = ["mse"]#["r2", "mse", "mae"]

#universal_best_li = 1
universal_best_model_features = "all_feats"

# Create a dummy plot for the legend
fig, ax = plt.subplots(figsize=(8, 1))  # Adjust width for a horizontal layout

# Add dummy lines for the legend
for k in all_k_values:
    ax.plot([], [], label=f"k={k}", marker="o", alpha=1, markersize=30, linewidth=4)

legend = ax.legend(
    loc='center', 
    ncol=len(all_k_values), 
    frameon=False,
    fontsize=48,
    handletextpad=0.3,  # Adjust padding between handle and text
    columnspacing=0.5,  # Adjust space between columns
    labelspacing=0.3    # Adjust vertical space between labels
)
ax.axis("off")

# Save the legend figure
legend_fig_path = f"{PLOTS_DIR}training_queries_legend_only.pdf"
fig.savefig(legend_fig_path, format="pdf", bbox_inches="tight")
print(f"Saved {legend_fig_path}")
plt.show()
plt.close(fig)  # Close the figure

# Iterate over datasets and metrics to generate and save figures
for ds_name in all_datasets:
    for metric in all_metrics:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        for k in all_k_values:
            best_li = str(dataset_params[ds_name]["li"])
            metric_values = [
                results[ds_name][k][s][str(best_li)][str(universal_best_model_features)][metric] 
                for s in all_training_queries
            ]
            
            if metric == "mse" or metric == "mae":
                metric_values = [v * 100 for v in metric_values]
                ax.set_ylabel(f" {metric_axis_titles[metric]} {r'$(10^{2})$'}")
            else:
                ax.set_ylabel(metric_axis_titles[metric])
                
            ax.plot(
                [int(q) for q in all_training_queries],
                metric_values,
                label=f"k={k}",
                marker="o",
                alpha=0.8,
                linewidth=3,
                markersize=23
            )
            
            for spine in ax.spines.values():
                spine.set_visible(False)
        

        ax.set_xlabel("# Training Queries")
        ax.grid(alpha=0.8, linestyle="--", linewidth=1)
        #ax.tick_params(axis='x', labelsize=44)
        output_path = f"{PLOTS_DIR}training_queries_{ds_name}_{metric}.pdf"
        fig.tight_layout()
        fig.savefig(output_path, format="pdf", bbox_inches="tight")
        if metric == "mse":
            print(f"Saved {output_path}")
            plt.show()
        plt.close(fig)

## Heuristic Parameter Generation for DARTH

### Recommendations for Naive Baseline

In [ ]:
# Reccomend the parameters for the early termination naive baseline:
all_datasets =  ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = ["10", "25", "50", "75", "100"]
all_r_targets = ["0.8", "0.85", "0.9", "0.95", "0.99"]

for ds_name in all_datasets:
    data_stat_filename = f"../../experiments/generated_json/data_stats_{ds_name}.json"
    data_stats = {}
    with open(data_stat_filename, "r") as f:
        data_stats = json.load(f)
    
    for k in all_k_values:
        
        conf_str = f"{ds_name}_tuples_k{k}=(\n"
        for r_target in all_r_targets:
            avg_dists = data_stats[k]["distance_calcs_to_reach_target_recall"][r_target]["avg"]
            reccomended_stop_point = avg_dists
            conf_str += f"  \"{float(r_target):.2f} {reccomended_stop_point:.0f}\" \n"
        conf_str += ")"
        print(conf_str, "\n")


### Recommendations for DARTH Heuristic methods (evaluations)

In [ ]:
all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = ["50"]
all_r_targets = ["0.90", "0.95", "0.99"]

ipi_div_factors = [2, 4]
mpi_div_factors = [4, 8, 10]

heuristic_recommendations_params = {}
for ipi_div in ipi_div_factors:
    for mpi_div in mpi_div_factors:
        if ipi_div > mpi_div:
            continue
        
        heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"] = {}
        for ds_name in all_datasets:
            heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name] = {}
            data_stat_filename = f"../../experiments/generated_json/data_stats_{ds_name}.json"
            data_stats = {}
            with open(data_stat_filename, "r") as f:
                data_stats = json.load(f)
            
            for k in all_k_values:
                heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][k] = {}
                for r_target in all_r_targets:
                    avg_dists = data_stats[k]["distance_calcs_to_reach_target_recall"][f"{float(r_target):.1f}"]["avg"]
                    reccomended_ipi = avg_dists / ipi_div
                    recommended_mpi = avg_dists / mpi_div
                    heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][k][r_target] = {
                        "ipi": f"{reccomended_ipi:.0f}",
                        "mpi": f"{recommended_mpi:.0f}"
                    }
        
with open(f"../../experiments/generated_json/heuristic_recommendations_params.json", "w") as f:
    json.dump(heuristic_recommendations_params, f, indent=4)

# Print the recommendations in batch format
for ds_name in all_datasets:
    for k in all_k_values:
        conf_str = f"{ds_name}_tuples_k{k}=(\n"
        for r_target in all_r_targets:
            for ipi_div in ipi_div_factors:
                for mpi_div in mpi_div_factors:
                    if ipi_div > mpi_div:
                        continue
                    recommended_ipi = heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][k][r_target]["ipi"]
                    recommended_mpi = heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][k][r_target]["mpi"]
                    conf_str += f"  \"{float(r_target):.2f} {recommended_ipi} {recommended_mpi}\" \n"
        conf_str += ")"
        
        print(conf_str, "\n")

In [ ]:
# Compare here all the versions of the recommendations
heuristic_recommendations_params

accepted_adaptive_heuristic_recommendations = {}
accepted_static_heuristic_recommendations = {}

#mpi_div_factors = [1, 2, 4, 8, 10, 16, 20]
#ipi_div_factors =  [1, 2, 4, 8]
ipi_div_factors = [2, 4]
mpi_div_factors = [4, 8, 10]

all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M"]
all_k_values = [50]
all_r_targets = ["0.90", "0.99"]

# Calculate the pair of mpi-ipi that has the
mpi_ipi_avg_speedups = {}
for mpi_div in mpi_div_factors:
    for ipi_div in ipi_div_factors:
        if ipi_div > mpi_div:
            continue
        
        ds_speedups = []
        for ds_name in all_datasets:
            M = dataset_params[ds_name]["M"]
            efC = dataset_params[ds_name]["efC"]
            efS = dataset_params[ds_name]["efS"]
            ds_k_speedups = []
            for k in all_k_values:
                ds_k_r_speedups = []
                for r_target in all_r_targets:
                    mpi = heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][str(k)][r_target]["mpi"]
                    ipi = heuristic_recommendations_params[f"{ipi_div}-{mpi_div}"][ds_name][str(k)][r_target]["ipi"]

                    testing_df = pd.read_csv(get_interval_tuning_dataset_name(M, efC, efS, 1000, ds_name, k, float(r_target), ipi, mpi, "heuristics"))
                    testing_df = testing_df[["qid", "step", "dists", "inserts", "elaps_ms", "r_actual", "r_predicted", "r_predictor_calls", "r_predictor_time_ms",]]
                    
                    baseline_df = pd.read_csv(get_no_early_stop_dataset_name(M, efC, efS, 1000, ds_name, k))
                    baseline_df = baseline_df[["qid", "step", "dists", "inserts", "elaps_ms", "r"]]
                    
                    speedup = baseline_df["elaps_ms"].mean() / testing_df["elaps_ms"].mean()
                    
                    ds_k_r_speedups.append(speedup)
                ds_k_speedups.append(np.mean(ds_k_r_speedups))
            ds_speedups.append(np.mean(ds_k_speedups))
        
        mpi_ipi_avg_speedups[f"{ipi_div}-{mpi_div}"] = np.mean(ds_speedups)
                   
# Sort by the mean speedup
sorted_mpi_ipi_avg_speedups = {k: v for k, v in sorted(mpi_ipi_avg_speedups.items(), key=lambda item: item[1], reverse=True)}

# Print all nicely: ipi: XX mpi: XX speedup: XX
for i, (ipi_mpi, speedup) in enumerate(sorted_mpi_ipi_avg_speedups.items()):
    print(f"(ipiDIV_mpiDIV)={ipi_mpi}: {speedup:.2f}x")
        

In [ ]:
# Generate the parameters
all_datasets = ["T2I100M", "SIFT100M", "GLOVE100", "GIST1M", "DEEP100M"]
all_k_values = ["10", "25", "50", "75", "100"]
all_r_targets = ["0.80", "0.85" ,"0.90", "0.95", "0.99"]

mpi_div_factor = 10
ipi_div_factor = 2

heuristic_recommendations_params = {}
for ds_name in all_datasets:
    heuristic_recommendations_params[ds_name] = {}
    data_stat_filename = f"../../experiments/generated_json/data_stats_{ds_name}.json"
    data_stats = {}
    with open(data_stat_filename, "r") as f:
        data_stats = json.load(f)
    
    for k in all_k_values:
        
        heuristic_recommendations_params[ds_name][k] = {}
        conf_str = f"{ds_name}_tuples_k{k}=(\n"
        for r_target in all_r_targets:
            avg_dists = data_stats[k]["distance_calcs_to_reach_target_recall"][f"{float(r_target):.1f}"]["avg"]
            reccomended_ipi = avg_dists / ipi_div_factor
            recommended_mpi = avg_dists / mpi_div_factor
            conf_str += f"  \"{float(r_target):.2f} {reccomended_ipi:.0f} {recommended_mpi:.0f}\" \n"
            heuristic_recommendations_params[ds_name][k][r_target] = {
                "ipi": f"{reccomended_ipi:.0f}",
                "mpi": f"{recommended_mpi:.0f}"
            }
        
        conf_str += ")"
        print(conf_str, "\n") 

# Save the recommendations to a file
mode = "adaptive"
if mpi_div_factor == ipi_div_factor:
    mode = "static"
with open(f"../../experiments/final_heuristic_{mode}_recommendations_params_ipidiv{ipi_div_factor}_mpidiv{mpi_div_factor}.json", "w") as f:
    json.dump(heuristic_recommendations_params, f, indent=4)
        

## Hyperparameter Tuning Comparisons and Ablation Study

In [ ]:
# Grid search results
initial_prediction_intervals_normal = [400, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
min_prediction_intervals_normal = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900]

initial_prediction_intervals_t2i = [400, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000]
min_prediction_intervals_t2i = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500]

target_recalls = [0.90, 0.99]
accepted_tuning_confs = {}
best_tuning_confs = {}

all_datasets = ["T2I100M", "SIFT100M", "GLOVE100", "GIST1M", "DEEP100M"]
all_k_values = [50]

for ds_name in all_datasets:
    accepted_tuning_confs[ds_name] = {}
    best_tuning_confs[ds_name] = {}
    
    M = dataset_params[ds_name]["M"]
    efC = dataset_params[ds_name]["efC"]
    efS = dataset_params[ds_name]["efS"]
    
    for k in all_k_values:
        best_tuning_confs[ds_name][k] = {
            "r_targets": [],
            "pi_s": []
        }
                
        accepted_tuning_confs[ds_name][k] = {}
        
        for target_recall in target_recalls:            
            best_speedup = 0
            best_mpi = 0
            best_ipi = 0
            best_target_recall = 0
            best_num_queries_under_target_recall = 0
            best_average_predictor_calls = 0
            
            if target_recall == 0.99 and ds_name == "T2I100M":
                target_recall = 0.95
            
            accepted_tuning_confs[ds_name][k][target_recall] = []
            
            #normal_intervals = (initial_prediction_intervals_normal, min_prediction_intervals_normal)
            #t2i_intervals = (initial_prediction_intervals_t2i, min_prediction_intervals_t2i)
            #for intervals in [normal_intervals, t2i_intervals]:
            
            if ds_name == "T2I100M":
                initial_prediction_intervals = initial_prediction_intervals_t2i
                min_prediction_intervals = min_prediction_intervals_t2i
            else:
                initial_prediction_intervals = initial_prediction_intervals_normal
                min_prediction_intervals = min_prediction_intervals_normal
            
            for ipi in initial_prediction_intervals:
                for mpi in min_prediction_intervals:
                    if (ipi < mpi):
                        continue
                                 
                    testing_df = pd.read_csv(get_interval_tuning_dataset_name(M, efC, efS, 1000, ds_name, k, target_recall, ipi, mpi, "search"))
                    testing_df = testing_df[["qid", "step", "dists", "inserts", "elaps_ms", "r_actual", "r_predicted", "r_predictor_calls", "r_predictor_time_ms",]]
                        
                    baseline_df = pd.read_csv(get_no_early_stop_dataset_name(M, efC, efS, 1000, ds_name, k))
                    baseline_df = baseline_df[["qid", "step", "dists", "inserts", "elaps_ms", "r"]]
                        
                    baseline_df = baseline_df[baseline_df["r"] >= float(target_recall)]
                    testing_df = testing_df[testing_df["qid"].isin(baseline_df["qid"])]
                        
                    avg_r_actual = testing_df["r_actual"].mean()
                    speedup = baseline_df["elaps_ms"].mean() / testing_df["elaps_ms"].mean()
                        
                    num_queries_under_target_recall = testing_df[testing_df["r_actual"] < (target_recall - 0.01)].shape[0]
                    average_predictor_calls = testing_df["r_predictor_calls"].mean()
                                            
                    if speedup > best_speedup and avg_r_actual >= target_recall:
                        best_speedup = speedup
                        best_ipi = ipi
                        best_mpi = mpi
                        best_target_recall = avg_r_actual
                        best_num_queries_under_target_recall = num_queries_under_target_recall
                        best_average_predictor_calls = average_predictor_calls
                        
                    if avg_r_actual >= target_recall - 0.01 and speedup >= 1.0:
                        entry = {
                                "speedup": speedup,
                                "ipi": ipi,
                                "mpi": mpi,
                                "recall": avg_r_actual,
                                "rqut": num_queries_under_target_recall,
                                "time": testing_df["elaps_ms"].mean(),
                        }
                        accepted_tuning_confs[ds_name][k][target_recall].append(entry)
                        #else:
                            #print(f"        Discarded tuning conf IPI,MPI: ({ipi},{mpi}) that reached {avg_r_actual:.2f} recall with speedup {speedup:.2f}x")
                        #    pass
                        #results[(ds_name, k)].append((ipi, mpi, avg_r_actual, speedup))
                    
                    
                    
            #print(f"    Rt={target_recall:.2f}: Speedup is {best_speedup:.2f}x with IPI={best_ipi}, MPI={best_mpi}, recall={best_target_recall:.2f}, QUT={best_num_queries_under_target_recall}, AVG Calls={best_average_predictor_calls}")     
            
            best_tuning_confs[ds_name][k]["r_targets"].append(target_recall)
            best_tuning_confs[ds_name][k]["pi_s"].append([best_ipi, best_mpi])
    print()       

sorted_accepted_tuning_confs = {}
for ds_name in accepted_tuning_confs.keys():
    sorted_accepted_tuning_confs[ds_name] = {}
    for k in accepted_tuning_confs[ds_name].keys():
        sorted_accepted_tuning_confs[ds_name][k] = {}
        for target_recall in accepted_tuning_confs[ds_name][k].keys():
            if target_recall == 0.99 and ds_name == "T2I100M":
                target_recall = 0.95
            sorted_accepted_tuning_confs[ds_name][k][target_recall] = sorted(accepted_tuning_confs[ds_name][k][target_recall], key=lambda x: x["speedup"], reverse=True)

DARTH_search_confs = {}
for ds_name in all_datasets:
    DARTH_search_confs[ds_name] = {}
    for k in all_k_values:
        DARTH_search_confs[ds_name][k] = {}
        print(f"{ds_name}, k={k}")
        for target_recall in target_recalls:
            if target_recall == 0.99 and ds_name == "T2I100M":
                target_recall = 0.95
            entry = sorted_accepted_tuning_confs[ds_name][k][target_recall][0]
            DARTH_search_confs[ds_name][k][target_recall] = entry
            print(f"    Rt={target_recall:.2f} => Speedup: {entry['speedup']:.2f}x | IPI: {entry['ipi']} | MPI: {entry['mpi']} | Recall: {entry['recall']:.2f} | QUT: {entry['rqut']}")
            

In [ ]:
# Calculate recall, speedups etc for the static prediction interval method

all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = [50]
target_recalls = [0.90, 0.99]
prediction_intervals_normal = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
prediction_intervals_t2i = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000]

static_prediction_interval_results = {}
for ds_name in all_datasets:
    static_prediction_interval_results[ds_name] = {}
    M = dataset_params[ds_name]["M"]
    efC = dataset_params[ds_name]["efC"]
    efS = dataset_params[ds_name]["efS"]
    for k in all_k_values:
        static_prediction_interval_results[ds_name][k] = {}
        for target_recall in target_recalls:
            if ds_name == "T2I100M" and target_recall == 0.99:
                target_recall = 0.95
            
            static_prediction_interval_results[ds_name][k][target_recall] = []
            
            prediction_intervals = prediction_intervals_normal
            if ds_name == "T2I100M":
                prediction_intervals = prediction_intervals_t2i
            
            for pi in prediction_intervals:
                mpi, ipi = pi, pi
 
                testing_df = pd.read_csv(get_interval_tuning_dataset_name(M, efC, efS, 1000, ds_name, k, target_recall, ipi, mpi, "static"))
                testing_df = testing_df[["qid", "step", "dists", "inserts", "elaps_ms", "r_actual", "r_predicted", "r_predictor_calls", "r_predictor_time_ms",]]
                    
                baseline_df = pd.read_csv(get_no_early_stop_dataset_name(M, efC, efS, 1000, ds_name, k))
                baseline_df = baseline_df[["qid", "step", "dists", "inserts", "elaps_ms", "r"]]
                
                baseline_df = baseline_df[baseline_df["r"] >= float(target_recall)]
                testing_df = testing_df[testing_df["qid"].isin(baseline_df["qid"])]
                    
                avg_r_actual = testing_df["r_actual"].mean()
                speedup = baseline_df["elaps_ms"].mean() / testing_df["elaps_ms"].mean()
                    
                num_queries_under_target_recall = testing_df[testing_df["r_actual"] < (target_recall - 0.01)].shape[0]
                average_predictor_calls = testing_df["r_predictor_calls"].mean()
                    
                if avg_r_actual >= target_recall - 0.01:
                    entry = {
                        "speedup": speedup,
                        "ipi": ipi,
                        "mpi": mpi,
                        "recall": avg_r_actual,
                        "rqut": num_queries_under_target_recall,
                        "time": testing_df["elaps_ms"].mean()
                    }
                    static_prediction_interval_results[ds_name][k][target_recall].append(entry)

# Sort the results
sorted_static_prediction_interval_results = {}
for ds_name in static_prediction_interval_results.keys():
    sorted_static_prediction_interval_results[ds_name] = {}
    for k in static_prediction_interval_results[ds_name].keys():
        sorted_static_prediction_interval_results[ds_name][k] = {}
        for target_recall in static_prediction_interval_results[ds_name][k].keys():
            if ds_name == "T2I100M" and target_recall == 0.99:
                target_recall = 0.95
            sorted_static_prediction_interval_results[ds_name][k][target_recall] = sorted(static_prediction_interval_results[ds_name][k][target_recall], key=lambda x: x["speedup"], reverse=True)
                    
# Print the best results
DARTH_static_confs = {}
for ds_name in all_datasets:
    for k in all_k_values:
        print(f"{ds_name}, k={k}")
        for target_recall in target_recalls:
            if ds_name == "T2I100M" and target_recall == 0.99:
                target_recall = 0.95
            entry = sorted_static_prediction_interval_results[ds_name][k][target_recall][0]
            print(f"    Rt={target_recall:.2f} => Speedup: {entry['speedup']:.2f}x | IPI: {entry['ipi']} | MPI: {entry['mpi']} | Recall: {entry['recall']:.2f} | QUT: {entry['rqut']}")
            DARTH_static_confs[(ds_name, k, target_recall)] = entry


In [ ]:
# Get the resutls ffrom the heuristic recommendations
DARTH_heuristic_adaptive_confs = {}
DARTH_heuristic_static_confs = {}

heuristic_adaptive_filename = "../../experiments/generated_json/final_heuristic_adaptive_recommendations_params_ipidiv2_mpidiv10.json"
heuristic_static_filename = "../../experiments/generated_json/final_heuristic_static_recommendations_params_ipidiv4_mpidiv4.json"

all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = [50]
target_recalls = [0.90, 0.99]

confs = [DARTH_heuristic_adaptive_confs, DARTH_heuristic_static_confs]
filenames = [heuristic_adaptive_filename, heuristic_static_filename]
modes = ["adaptive", "static"]

for DARTH_conf, filename, mode in zip(confs, filenames, modes):
    heuristic_recommendations_params = {}
    with open(filename, "r") as f:
        heuristic_recommendations_params = json.load(f)

    print(f"Processing {mode} recommendations")
    
    for ds_name in all_datasets:
        DARTH_conf[ds_name] = {}
        M = dataset_params[ds_name]["M"]
        efC = dataset_params[ds_name]["efC"]
        efS = dataset_params[ds_name]["efS"]
        for k in all_k_values:
            print(f"    {ds_name}, k={k}")
            DARTH_conf[ds_name][k] = {}
            for target_recall in target_recalls:
                if ds_name == "T2I100M" and target_recall == 0.99:
                    target_recall = 0.95
                
                mpi = heuristic_recommendations_params[ds_name][str(k)][str(f"{float(target_recall):.2f}")]["mpi"]
                ipi = heuristic_recommendations_params[ds_name][str(k)][str(f"{float(target_recall):.2f}")]["ipi"]
                
                print(f"        Rt={target_recall:.2f} => IPI: {ipi} | MPI: {mpi}")
                
                testing_df = pd.read_csv(get_interval_tuning_dataset_name(M, efC, efS, 1000, ds_name, k, target_recall, ipi, mpi, "heuristics"))
                testing_df = testing_df[["qid", "step", "dists", "inserts", "elaps_ms", "r_actual", "r_predicted", "r_predictor_calls", "r_predictor_time_ms",]]
                
                baseline_df = pd.read_csv(get_no_early_stop_dataset_name(M, efC, efS, 1000, ds_name, k))
                baseline_df = baseline_df[["qid", "step", "dists", "inserts", "elaps_ms", "r"]]
                
                baseline_df = baseline_df[baseline_df["r"] >= float(target_recall)]
                testing_df = testing_df[testing_df["qid"].isin(baseline_df["qid"])]
                
                avg_r_actual = testing_df["r_actual"].mean()
                speedup = baseline_df["elaps_ms"].mean() / testing_df["elaps_ms"].mean()
                
                num_queries_under_target_recall = testing_df[testing_df["r_actual"] < (target_recall - 0.01)].shape[0]
                average_predictor_calls = testing_df["r_predictor_calls"].mean()
                                
                DARTH_conf[ds_name][k][target_recall] = {
                    "speedup": speedup,
                    "ipi": int(ipi),
                    "mpi": int(mpi),
                    "recall": avg_r_actual,
                    "rqut": num_queries_under_target_recall,
                    "time": testing_df["elaps_ms"].mean()
                }

In [ ]:
DARTH_heuristic_adaptive_confs["T2I100M"][k][0.95]

In [ ]:
from matplotlib.patches import Patch

all_datasets = ["SIFT100M", "DEEP100M", "GLOVE100", "GIST1M"]
all_k_values = [50]

algos = {
    "Static-Grid-Search": {
        "color": "peachpuff",
    },
    "Adaptive-Grid-Search": {
        "color": "darksalmon",
    },
    "Static-Heuristic": {
        "color": "burlywood",
    },
    "Adaptive-Heuristic": {
        "color": "tomato",
    }
}

plt.rcParams.update({"font.size": 41})

fig_legend, ax_legend = plt.subplots(figsize=(8, 1))  # Adjust the figsize for better alignment
legend_elements = [
    Patch(facecolor=algos["Static-Grid-Search"]["color"], edgecolor=algos["Static-Grid-Search"]["color"], label="St-GS"),
    Patch(facecolor=algos["Adaptive-Grid-Search"]["color"], edgecolor=algos["Adaptive-Grid-Search"]["color"], label="Ad-GS"),
    Patch(facecolor=algos["Static-Heuristic"]["color"], edgecolor=algos["Static-Heuristic"]["color"], label="St-Heur"),
    Patch(facecolor=algos["Adaptive-Heuristic"]["color"], edgecolor=algos["Adaptive-Heuristic"]["color"], label="Ad-Heur"),
]
#ax_legend.legend(handles=legend_elements, loc='center', ncol=4, frameon=False)
ax_legend.legend(
    handles=legend_elements, 
    loc='center', 
    ncol=4, 
    frameon=False,
    handletextpad=0.3,  # Adjust padding between handle and text
    columnspacing=0.5,  # Adjust space between columns
    labelspacing=0.3    # Adjust vertical space between labels
)
ax_legend.axis('off')
fig_legend.savefig(f"{PLOTS_DIR}interval_ablation_legend_only.pdf", bbox_inches="tight")


#plt.rcParams.update({"font.size": 36})  

measures = {
    "speedup": {
        "title": "Speedup",
        "label": "Times Faster",
        "y-bottom-lim": 1
    },
    #"time": {
    #    "title": "Time",
    #    "label": "Search Time (ms)",
    #    "y-bottom-lim": 0
    #}
    #"rqut": {
    #    "title": "Queries% Under Target Recall",
    #    "label": "RQUT",
    #    "y-bottom-lim": 0
    #    
    #}
}

#change colors
target_recalls = [0.90, 0.99]

# Calculate also the average speedup of grid search adaptive to compare with the heuristic adaptive
avg_speedup_grid_search_adaptive_over_heuristic = 0
avg_speedup_grid_search_adaptive_over_static = 0
avg_speedup_heuristic_adaptive_over_static_heuristic = 0
speedup_nums = 0

max_speedup_grid_search_adaptive_over_static = 0
max_speedup_heuristic_adaptive_over_static = 0

for k in all_k_values:
    for i, target_recall in enumerate(target_recalls):
        for measure, measure_info in measures.items():
            DARTH_heuristic_adaptive = []#[DARTH_heuristic_adaptive_confs[ds_name][k][target_recall][measure] for ds_name in all_datasets]
            DARTH_heuristic_static = []#[DARTH_heuristic_static_confs[ds_name][k][target_recall][measure] for ds_name in all_datasets]
            DARTH_grid_search_adaptive = []#[DARTH_search_confs[ds_name][k][target_recall][measure] for ds_name in all_datasets]
            DARTH_grid_search_static = []#[DARTH_static_confs[(ds_name, k, target_recall)][measure] for ds_name in all_datasets]
            for ds_name in all_datasets:
                prev_recall = target_recall
                if ds_name == "T2I100M" and target_recall == 0.99:
                    target_recall = 0.95
                
                DARTH_heuristic_adaptive.append(DARTH_heuristic_adaptive_confs[ds_name][k][target_recall][measure])
                DARTH_heuristic_static.append(DARTH_heuristic_static_confs[ds_name][k][target_recall][measure])
                DARTH_grid_search_adaptive.append(DARTH_search_confs[ds_name][k][target_recall][measure])
                DARTH_grid_search_static.append(DARTH_static_confs[(ds_name, k, target_recall)][measure])

                time_heuristic_adaptive = DARTH_heuristic_adaptive_confs[ds_name][k][target_recall]["time"]
                time_grid_search_adaptive = DARTH_search_confs[ds_name][k][target_recall]["time"]
                time_heuristic_static = DARTH_heuristic_static_confs[ds_name][k][target_recall]["time"]
                time_grid_search_static = DARTH_static_confs[(ds_name, k, target_recall)]["time"]
                
                avg_speedup_grid_search_adaptive_over_heuristic += time_heuristic_adaptive / time_grid_search_adaptive
                avg_speedup_grid_search_adaptive_over_static += time_heuristic_static / time_grid_search_adaptive
                avg_speedup_heuristic_adaptive_over_static_heuristic += time_heuristic_static / time_heuristic_adaptive
                
                speedup_nums += 1
                
                speedup_grid_search_adaptive_vs_static = time_grid_search_static / time_grid_search_adaptive
                if speedup_grid_search_adaptive_vs_static > max_speedup_grid_search_adaptive_over_static:
                    max_speedup_grid_search_adaptive_over_static = speedup_grid_search_adaptive_vs_static
                    
                speedup_heuristic_adaptive_vs_static = time_heuristic_static / time_heuristic_adaptive
                if speedup_heuristic_adaptive_vs_static > max_speedup_heuristic_adaptive_over_static:
                    max_speedup_heuristic_adaptive_over_static = speedup_heuristic_adaptive_vs_static
                
                target_recall = prev_recall
            
            fig, ax = plt.subplots(1, 1, figsize=(10,6))
            
            x = np.arange(len(all_datasets))
            width = 0.2
            
            ax.bar(x - 1.5 * width, DARTH_grid_search_static, width, label="Static-Grid-Search", alpha=0.8, color=algos["Static-Grid-Search"]["color"], edgecolor="black", linewidth=2)
            ax.bar(x - 0.5 * width, DARTH_grid_search_adaptive, width, label="Adaptive-Grid-Search", alpha=0.8, color=algos["Adaptive-Grid-Search"]["color"], edgecolor="black", linewidth=2)
            ax.bar(x + 0.5 * width, DARTH_heuristic_static, width, label="Static-Heuristic", alpha=0.8,color=algos["Static-Heuristic"]["color"], edgecolor="black", linewidth=2)
            ax.bar(x + 1.5 * width, DARTH_heuristic_adaptive, width, label="Adaptive-Heuristic", alpha=0.8, color=algos["Adaptive-Heuristic"]["color"], edgecolor="black", linewidth=2)
            
            ax.set_xticks(x)
            ax.set_xticklabels(dataset_params[ds_namee]["label"] for ds_namee in all_datasets)
            
            for spine in ax.spines.values():
                spine.set_visible(False)
                
            # Turn x-axis labels to 45 degrees
            plt.xticks(rotation=15)
            #plt.yticks(fontsize=36)
            
            # Make the font size of x-axis labels smaller
            plt.xticks(fontsize=36)
            #plt.yticks(fontsize=30)
            # make ylabel smaller
            
            
            # change y name font size 
            ax.set_ylabel(f"{measure_info['label']}")#, fontsize=28)
            ax.grid(alpha=0.8, axis="y", linestyle="--", linewidth=1.0)
            ax.set_ylim(bottom=measure_info["y-bottom-lim"])
        
            fig.tight_layout()
            fig.savefig(f"{PLOTS_DIR}interval_ablation_{measure}_k{k}_rt{target_recall:.2f}.pdf", bbox_inches="tight")
            print(f"Saved plot at {PLOTS_DIR}interval_ablation_{measure}_k{k}_rt{target_recall:.2f}.pdf")
            plt.show()

avg_speedup_grid_search_adaptive_over_heuristic /= speedup_nums
avg_speedup_grid_search_adaptive_over_static /= speedup_nums
avg_speedup_heuristic_adaptive_over_static_heuristic /= speedup_nums

print(f"Average speedup of Grid-Search Adaptive over Heuristic Adaptive: {avg_speedup_grid_search_adaptive_over_heuristic:.2f}x")
print(f"Average speedup of Grid-Search Adaptive over Static: {avg_speedup_grid_search_adaptive_over_static:.2f}x")
print(f"Average speedup of Heuristic Adaptive over Static Heuristic: {avg_speedup_heuristic_adaptive_over_static_heuristic:.2f}x")

print(f"Max speedup of Grid-Search Adaptive over Static: {max_speedup_grid_search_adaptive_over_static:.2f}x")
print(f"Max speedup of Heuristic Adaptive over Static: {max_speedup_heuristic_adaptive_over_static:.2f}x")


## Tuned parameter organization for LAET and Classic HNSW

### Generate tuned multipliers of LAET

In [ ]:
laet_dataset_F_params = {
    "SIFT100M": 241,
    "GLOVE100": 200,
    "GIST1M": 1260,
    "DEEP100M": 368,
    "T2I100M": 400,
}

laet_multipliers_conf = {}

with open("../../experiments/generated_json/laet_tuning_results_memoryFalse_validationSize1000.json", "r") as f:
    laet_multipliers_conf = json.load(f)

all_datasets = ["T2I100M","SIFT100M", "GLOVE100", "GIST1M", "DEEP100M"]
all_k_values = ["10", "25", "50", "75", "100"]
all_r_targets = ["0.8", "0.85", "0.9", "0.95", "0.99"]

for ds_name in all_datasets:
    for k in all_k_values:
        str_conf = f"{ds_name}_tuples_k{k}=(\n"
        for r_target in all_r_targets:
            F = laet_dataset_F_params[ds_name]
            if ds_name == "T2I100M" and r_target == "0.99":
                continue
            multiplier_info = laet_multipliers_conf[ds_name][k][r_target] # Make sure to see if this is correct
            multiplier = multiplier_info["min_m"]
            str_conf += f"  \"{float(r_target):.2f} {F} {float(multiplier):.2f}\" \n"
        str_conf += ")"
        print(str_conf, "\n")

### Generate tuned multipliers of Classic HNSW

In [ ]:
classic_hnsw_conf = {}
with open("../../experiments/generated_json/classic_hnsw_tuning_results_memoryFalse_validationSize1000.json", "r") as f:
    classic_hnsw_conf = json.load(f)
    
all_datasets = ["T2I100M","SIFT100M", "GLOVE100", "GIST1M", "DEEP100M"]
all_k_values = ["10", "25", "50", "75", "100"]
all_r_targets = ["0.8", "0.85", "0.9", "0.95", "0.99"]

for ds_name in all_datasets:
    for k in all_k_values:
        str_conf = f"{ds_name}_tuples_k{k}=(\n"
        for r_target in all_r_targets:
            if ds_name == "T2I100M" and r_target == "0.99":
                continue
            efS = classic_hnsw_conf[ds_name][k][r_target]["min_efS"]
            str_conf += f"  \"{float(r_target):.2f} {efS}\" \n"
        str_conf += ")"
        print(str_conf, "\n")

## Index Results

In [ ]:
all_datasets = ["SIFT100M", "GLOVE100", "GIST1M", "DEEP100M", "T2I100M"]
all_k_values = ["10", "25", "50", "75", "100"]

for ds_name in all_datasets:
    M = dataset_params[ds_name]["M"]
    efC = dataset_params[ds_name]["efC"]
    efS = dataset_params[ds_name]["efS"]
    
    print(f"{ds_name}")
    
    avg_recall_k = []
    for k in all_k_values:        
        no_early_termination_df = pd.read_csv(get_no_early_stop_dataset_name(M, efC, efS, 1000, ds_name, k))
        recall = no_early_termination_df["r"].mean()
        avg_recall_k.append(recall)
        print(f"        k={k} => {recall:.3f}")
    
    print(f"    Avg Recall: {np.mean(avg_recall_k):.3f}")
